# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

Date      id Warehouse    Shipments             Customer  \
0  Apr 18 2023  4:51PM  261373        22       646170    NBTY Global, Inc.   
1  Apr 18 2023  4:32PM  261370        22       646151    NBTY Global, Inc.   
2  Apr 18 2023  4:30PM  261368        12      9393334     SD Head USA, LLC   
3  Apr 18 2023  3:04PM  261354        19  ADV80016035  AdvaGen Pharma, Ltd   
4  Apr 18 2023  3:04PM  261354        19  ADV80016036  AdvaGen Pharma, Ltd   
5  Apr 18 2023  3:04PM  261354        19  ADV80016037  AdvaGen Pharma, Ltd   
6  Apr 18 2023  3:04PM  261354        19  ADV80016038  AdvaGen Pharma, Ltd   
7  Apr 18 2023  3:04PM  261354        19  ADV80016039  AdvaGen Pharma, Ltd   
8  Apr 18 2023  3:04PM  261354        19  ADV80016040  AdvaGen Pharma, Ltd   
9  Apr 18 2023  3:04PM  261354        19  ADV80016041  AdvaGen Pharma, Ltd   

  ShipmentStatus  
0       Released  
1       Released  
2       Released  
3      Executing  
4       Released  
5      Executing  
6      Executing  
7      Executing  
8      Executing  
9      Completed

In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

id ShipmentStatus  Shipments
8   261354      Executing         10
9   261354       Released          1
10  261368       Released          1
11  261370       Released          1
12  261373       Released          1

In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus  Completed  Executing  Released
id                                            
261351                NaN        NaN       1.0
261354                4.0       10.0       1.0
261368                NaN        NaN       1.0
261370                NaN        NaN       1.0
261373                NaN        NaN       1.0

In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus  Completed  Executing  Released
id                                            
261273               11.0        0.0       1.0
261300                0.0        0.0       1.0
261331                0.0        1.0       0.0
261334                0.0        0.0      27.0
261343                0.0        0.0       1.0

In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus  Completed  Executing  Released
id                                            
261273                 11          0         1
261300                  0          0         1
261331                  0          1         0
261334                  0          0        27
261343                  0          0         1

In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus      id  Completed  Executing  Released
0               261273         11          0         1
1               261300          0          0         1
2               261331          0          1         0
3               261334          0          0        27
4               261343          0          0         1

In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus      id Completed Executing Released
0               261273        11                  1
1               261300                            1
2               261331                   1         
3               261334                           27
4               261343                            1

In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

Date      id Warehouse                            Customer
0   Apr 18 2023  4:51PM  261373        22                   NBTY Global, Inc.
1   Apr 18 2023  4:32PM  261370        22                   NBTY Global, Inc.
2   Apr 18 2023  4:30PM  261368        12                    SD Head USA, LLC
3   Apr 18 2023  3:04PM  261354        19                 AdvaGen Pharma, Ltd
18  Apr 18 2023  3:00PM  261351        10                   ISDIN Corporation
19  Apr 18 2023  1:25PM  261343        19              NAPP Technologies, LLC
20  Apr 18 2023 12:13PM  261334        19              GCH Granules USA, Inc.
47  Apr 18 2023 12:05PM  261331        10  Uniderm USA, Inc - Collagenil Line
48  Apr 18 2023 10:01AM  261300        20           Andelyn Biosciences, Inc.
49  Apr 18 2023  7:57AM  261273        19                 AdvaGen Pharma, Ltd

In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

Date      id Warehouse                            Customer  \
0  Apr 18 2023  4:51PM  261373        22                   NBTY Global, Inc.   
1  Apr 18 2023  4:32PM  261370        22                   NBTY Global, Inc.   
2  Apr 18 2023  4:30PM  261368        12                    SD Head USA, LLC   
3  Apr 18 2023  3:04PM  261354        19                 AdvaGen Pharma, Ltd   
4  Apr 18 2023  3:00PM  261351        10                   ISDIN Corporation   
5  Apr 18 2023  1:25PM  261343        19              NAPP Technologies, LLC   
6  Apr 18 2023 12:13PM  261334        19              GCH Granules USA, Inc.   
7  Apr 18 2023 12:05PM  261331        10  Uniderm USA, Inc - Collagenil Line   
8  Apr 18 2023 10:01AM  261300        20           Andelyn Biosciences, Inc.   
9  Apr 18 2023  7:57AM  261273        19                 AdvaGen Pharma, Ltd   

  Completed Executing Released  
0                            1  
1                            1  
2                            1  
3         4        10        1  
4                            1  
5                            1  
6                           27  
7                   1           
8                            1  
9        11                  1

In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

Date      id Warehouse                            Customer  \
0  Apr 18 2023  4:51PM  261373        22                   NBTY Global, Inc.   
1  Apr 18 2023  4:32PM  261370        22                   NBTY Global, Inc.   
2  Apr 18 2023  4:30PM  261368        12                    SD Head USA, LLC   
3  Apr 18 2023  3:04PM  261354        19                 AdvaGen Pharma, Ltd   
4  Apr 18 2023  3:00PM  261351        10                   ISDIN Corporation   
5  Apr 18 2023  1:25PM  261343        19              NAPP Technologies, LLC   
6  Apr 18 2023 12:13PM  261334        19              GCH Granules USA, Inc.   
7  Apr 18 2023 12:05PM  261331        10  Uniderm USA, Inc - Collagenil Line   
8  Apr 18 2023 10:01AM  261300        20           Andelyn Biosciences, Inc.   
9  Apr 18 2023  7:57AM  261273        19                 AdvaGen Pharma, Ltd   

  Released Executing Completed  
0        1                      
1        1                      
2        1                      
3        1        10         4  
4        1                      
5        1                      
6       27                      
7                  1            
8        1                      
9        1                  11

In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

Date      id Warehouse             Customer Released  \
0  Apr 18 2023  4:51PM  261373        22    NBTY Global, Inc.        1   
1  Apr 18 2023  4:32PM  261370        22    NBTY Global, Inc.        1   
2  Apr 18 2023  4:30PM  261368        12     SD Head USA, LLC        1   
3  Apr 18 2023  3:04PM  261354        19  AdvaGen Pharma, Ltd        1   
4  Apr 18 2023  3:00PM  261351        10    ISDIN Corporation        1   

  Executing Completed  
0                      
1                      
2                      
3        10         4  
4

In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Date      id Warehouse             Customer  Released  \
0  Apr 18 2023  4:51PM  261373        22    NBTY Global, Inc.       1.0   
1  Apr 18 2023  4:32PM  261370        22    NBTY Global, Inc.       1.0   
2  Apr 18 2023  4:30PM  261368        12     SD Head USA, LLC       1.0   
3  Apr 18 2023  3:04PM  261354        19  AdvaGen Pharma, Ltd       1.0   
4  Apr 18 2023  3:00PM  261351        10    ISDIN Corporation       1.0   

   Executing  Completed  
0        NaN        NaN  
1        NaN        NaN  
2        NaN        NaN  
3       10.0        4.0  
4        NaN        NaN

In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

Date      id Warehouse             Customer  Released  \
0  Apr 18 2023  4:51PM  261373        22    NBTY Global, Inc.       1.0   
1  Apr 18 2023  4:32PM  261370        22    NBTY Global, Inc.       1.0   
2  Apr 18 2023  4:30PM  261368        12     SD Head USA, LLC       1.0   
3  Apr 18 2023  3:04PM  261354        19  AdvaGen Pharma, Ltd       1.0   
4  Apr 18 2023  3:00PM  261351        10    ISDIN Corporation       1.0   

   Executing  Completed  
0        0.0        0.0  
1        0.0        0.0  
2        0.0        0.0  
3       10.0        4.0  
4        0.0        0.0

In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

id  Released  Executing  Completed
Warehouse                                         
10          522682       1.0        1.0        0.0
12          261368       1.0        0.0        0.0
19         1045304      30.0       10.0       15.0
20          261300       1.0        0.0        0.0
22          522743       2.0        0.0        0.0

In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

Warehouse       id  Released  Executing  Completed
0        10   522682       1.0        1.0        0.0
1        12   261368       1.0        0.0        0.0
2        19  1045304      30.0       10.0       15.0
3        20   261300       1.0        0.0        0.0
4        22   522743       2.0        0.0        0.0

In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

Warehouse  Released  Executing  Completed
0        10       1.0        1.0        0.0
1        12       1.0        0.0        0.0
2        19      30.0       10.0       15.0
3        20       1.0        0.0        0.0
4        22       2.0        0.0        0.0

In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

Warehouse    Status  Value
0        10  Released    1.0
1        12  Released    1.0
2        19  Released   30.0
3        20  Released    1.0
4        22  Released    2.0

In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse   10   12    19   20   22
Status                             
Completed  0.0  0.0  15.0  0.0  0.0
Executing  1.0  0.0  10.0  0.0  0.0
Released   1.0  1.0  30.0  1.0  2.0

In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse     Status   10   12    19   20   22
0          Completed  0.0  0.0  15.0  0.0  0.0
1          Executing  1.0  0.0  10.0  0.0  0.0
2           Released  1.0  1.0  30.0  1.0  2.0

In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

Status   10   12    19   20   22
0  Completed  0.0  0.0  15.0  0.0  0.0
1  Executing  1.0  0.0  10.0  0.0  0.0
2   Released  1.0  1.0  30.0  1.0  2.0

In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()